### Step 1: Install Dependencies
First, make sure you have MLflow and scikit-learn installed. You can install them using pip:

In [1]:
#!pip3 install mlflow scikit-learn

### Step 2: Import Libraries
In your Python script, import the necessary libraries:

In [1]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from urllib.parse import urlparse

import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
mlflow.autolog() 


2025/01/21 12:40:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/01/21 12:40:41 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2025/01/21 12:40:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.


### Step 3: Load and Prepare Data
Load a dataset from scikit-learn, split it into training and testing sets, and perform any necessary data preprocessing:

In [2]:
data = load_diabetes()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Step 4: Train and Log the Model
Train a simple linear regression model using scikit-learn and log it using MLflow:

In [3]:
with mlflow.start_run():
    # Create and train a linear regression model
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    predictions = lr.predict(X_train)
    signature = infer_signature(X_train, predictions)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":
        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        mlflow.sklearn.log_model(
            lr, "model", registered_model_name="RandomForest", signature=signature
        )
    else:
        mlflow.sklearn.log_model(lr, "model", signature=signature)

This code initializes an MLflow run, trains the model, and logs it with the name "model."

### Step 5: Save the MLflow Artifacts
By default, MLflow logs the model into an artifact store. You can specify the artifact location in your MLflow server configuration or use the default location. You can also set the environment variable MLFLOW_TRACKING_URI to point to your MLflow server.

### Step 6: Serve the Model
To serve the model using MLflow, you can use the mlflow models serve command. First, make sure that MLflow is running as a server (e.g., mlflow server).

Then, you can start serving your model:

### In case of Error

In [4]:
# https://github.com/pyenv/pyenv/wiki#suggested-build-environment

In [5]:
# !rm -rf '/home/ramin/.pyenv'

In [6]:
# !curl https://pyenv.run | bash
# !python -m  pip install virtualenv
# !PATH="$HOME/.pyenv/bin:$PATH"
# !export PYENV_ROOT="$HOME/.pyenv"

### In-case of dependeny error

In [ ]:
# mlflow.pyfunc.get_model_dependencies('mlruns/0/e6f6f958a20c407f9d485ba5fb24f6c2/artifacts/model')

In [ ]:
# !pip3 install -r mlruns/0/aea73ab36d544af29927cf26199b8888/artifacts/model/requirements.txt

### To run on a severate bash screen within the same venv

In [ ]:
!mlflow models serve --env-manager=local -m mlruns/0/1b22a8a9589a4adeaff189de8a358580/artifacts/model -h 127.0.0.1 -p 5001

### Step 7: Make Predictions
You can make predictions by sending HTTP POST requests to the model server. Here's an example using Python's requests library:

In [5]:
X_test.shape

(89, 10)

In [6]:
import requests
import json

url = 'http://127.0.0.1:5001/invocations'

data = {
    "columns": data['feature_names'],
    "instances": X_test.tolist()
}

response = requests.post(url, json=data)
predictions = response.json()

print(predictions)

{'predictions': [139.54755840379607, 179.51720835342778, 134.03875571890114, 291.4170292522082, 123.7896587223961, 92.17234650105041, 258.23238898921295, 181.33732057060732, 90.22411310941467, 108.63375858007933, 94.13865743638853, 168.4348635797548, 53.50478879609554, 206.6308165929643, 100.12925868961605, 130.6665708505271, 219.53071499182573, 250.78032340333357, 196.36883460186397, 218.57511814821788, 207.35050181974748, 88.48340941420321, 70.43285917378927, 188.95914235039012, 154.8868161968979, 159.36170121791906, 188.31263362697564, 180.39094032857133, 47.990465608700134, 108.97453871427219, 174.7789763289737, 86.36406655640144, 132.95761214676355, 184.53819482628649, 173.83220911339012, 190.35858491915616, 124.41561760139221, 119.65110656071295, 147.95168682418492, 59.054052414728076, 71.6233185615158, 107.68284703885513, 165.45365458042076, 155.00975931051346, 171.0479909563818, 61.457613561017894, 71.6667258072822, 114.96732205941937, 51.57975523042386, 167.5759952801152, 152.

The url variable, in the context of the code provided, represents the Uniform Resource Locator (URL) that is used to specify the location or address of a web resource. In this case, it points to a specific endpoint on a local web server:

* http:// indicates the protocol being used, which is Hypertext Transfer Protocol (HTTP). HTTP is the foundation of data communication on the World Wide Web.

* localhost is a special hostname that typically refers to the current device or computer where the code is running. It is often used to access services running on the same machine.

* 5000 is the port number. Ports are used to differentiate between different services or processes running on the same machine. Port 5000 is a commonly used port for running web servers locally.

* /invocations is the path or endpoint on the web server. In the context of machine learning deployment, this path often represents an API endpoint for making predictions using a deployed machine learning model. In this specific case, it's likely that the web server running on localhost:5000 has an endpoint called /invocations that allows you to send data to the deployed model and receive predictions in response.

So, the url variable is essentially storing the URL of the API endpoint where you can send a POST request with data to obtain predictions from a locally deployed machine learning model.